# CARLA Autonomous Vehicle with LiDAR-Camera Fusion

**AI-controlled vehicle using LiDAR-to-camera projection for obstacle detection and navigation**

Output: `output/YYYY-MM-DD_HH-MM-SS/lidar_projection/camera_{front,left,right}/`

## 1. Imports & Server Connection

In [1]:
import time
import numpy as np

from src import (
    Config, connect_to_carla, setup_synchronous_mode, restore_async_mode,
    clear_environment, reload_environment, spawn_vehicle, spawn_cameras,
    spawn_lidar, create_sensor_queues, project_lidar_to_camera,
    save_projection_image, lidar_density_steering, update_spectator_view,
)
from src.sensors import create_camera_callback, create_lidar_callback
from src.projection import image_to_array
from src.steering import get_steering_direction

config = Config()

# Connect to CARLA and enable synchronous mode
client, world = connect_to_carla(config)
original_settings, traffic_manager = setup_synchronous_mode(world, client, config)

print(f"✓ Connected to CARLA | Map: {world.get_map().name} | {config.fps} FPS sync mode")

✓ Connected to CARLA | Map: Carla/Maps/Town10HD_Opt | 20 FPS sync mode


---

## 2. Main Pipeline

Setup environment, spawn vehicle & sensors, run simulation with LiDAR-based steering.

In [5]:
# Environment setup (optional: clear/reload)
clear_environment(world)  # Remove buildings, props, etc.
# reload_environment(world)  # Uncomment to restore

# Spawn vehicle and sensors
vehicle = spawn_vehicle(world, config)
cameras, K = spawn_cameras(world, vehicle, config)
lidar = spawn_lidar(world, vehicle, config)
update_spectator_view(world, vehicle)
world.tick()

# Setup sensor queues and output directory
camera_queues, lidar_queue = create_sensor_queues(cameras)
output_dir = config.get_output_dir()
output_dirs = config.create_output_dirs(output_dir)

for cam_name, cam in cameras.items():
    cam.listen(create_camera_callback(camera_queues, cam_name))
lidar.listen(create_lidar_callback(lidar_queue))

print(f"✓ Vehicle + {len(cameras)} cameras + LiDAR spawned")
print(f"✓ Output: {output_dir}/")

✓ Vehicle + 3 cameras + LiDAR spawned
✓ Output: output/2026-01-27_13-49-12/


In [6]:
# Run simulation with LiDAR density-based steering
frame_counter, tick_count, last_print_time = 0, 0, 0
current_projection_counts = {}

print(f"✓ Running {config.simulation_duration}s | Throttle: {config.throttle} | Steering gain: {config.steering_gain}\n")

try:
    while tick_count < config.total_ticks:
        world.tick()
        tick_count += 1
        
        # Wait for sensor data
        start = time.time()
        while time.time() - start < 2.0:
            if all(camera_queues[c].qsize() > 0 for c in cameras) and lidar_queue.qsize() > 0:
                break
            time.sleep(0.001)
        
        should_save = (tick_count % config.capture_interval_ticks == 0)
        
        # Collect and clear queues
        camera_images = {n: camera_queues[n].get() for n in cameras if not camera_queues[n].empty()}
        for q in camera_queues.values():
            while not q.empty(): q.get()
        lidar_data = lidar_queue.get() if not lidar_queue.empty() else None
        while not lidar_queue.empty(): lidar_queue.get()
        
        # Process sensors
        if len(camera_images) == len(cameras) and lidar_data:
            projection_counts = {}
            for cam_name in cameras:
                proj_array, _, weighted_density = project_lidar_to_camera(
                    lidar_data, lidar, cameras[cam_name], image_to_array(camera_images[cam_name]), K, config)
                projection_counts[cam_name] = weighted_density
                if should_save:
                    save_projection_image(cam_name, proj_array, frame_counter, output_dirs)
            
            if should_save:
                frame_counter += 1
            current_projection_counts = projection_counts
        
        # Apply steering and update view
        if current_projection_counts:
            vehicle.apply_control(lidar_density_steering(current_projection_counts, config))
        update_spectator_view(world, vehicle)
        
        # Status every 5s
        sim_time = tick_count * config.fixed_delta_seconds
        if int(sim_time) % 5 == 0 and int(sim_time) != last_print_time:
            last_print_time = int(sim_time)
            vel = vehicle.get_velocity()
            speed = 3.6 * np.sqrt(vel.x**2 + vel.y**2 + vel.z**2)
            l, f, r = [current_projection_counts.get(k, 0) for k in ['left', 'front', 'right']]
            print(f"[{int(sim_time)}s] {speed:.1f} km/h | L:{l:.0f} F:{f:.0f} R:{r:.0f} → {get_steering_direction(current_projection_counts)}")

except KeyboardInterrupt:
    print("\n✗ Interrupted")

print(f"\n✓ Done! {frame_counter} frames saved")

✓ Running 30.0s | Throttle: 0.3 | Steering gain: 0.8

[5s] 0.1 km/h | L:17763 F:13857 R:19734 → STRAIGHT (front clear)
[10s] 0.0 km/h | L:17634 F:13799 R:19219 → STRAIGHT (front clear)

✗ Interrupted

✓ Done! 24 frames saved


---

## 3. Cleanup & Shutdown

Stop sensors, destroy actors, restore async mode.

In [4]:
restore_async_mode(world, original_settings, traffic_manager)

for cam in cameras.values():
    cam.stop()
lidar.stop()
time.sleep(0.5)

for cam in cameras.values():
    cam.destroy()
lidar.destroy()
vehicle.destroy()
world.tick()

print(f"✓ Cleanup done | Output: {output_dir}/lidar_projection/")

✓ Cleanup done | Output: output/2026-01-27_13-47-46/lidar_projection/
